#### Настройки с учетом версий библиотеки scikit-learn

In [1]:
# альтернативный синтаксис для sklearn 0.18
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version
if Version(sklearn_version) < '0.18':
    from sklearn.grid_search import train_test_split
else:
    from sklearn.model_selection import train_test_split

#### Импорт набора данных

In [4]:
import numpy as np
import pandas as pd
#from sklearn.impute import SimpleImputer 

url = 'mammographic_masses.data'
df = pd.read_csv(url, header=None)
df

,0,1,2,3,4,5
0,5,67,3,5,3,1
1,4,43,1,1,?,1
2,5,58,4,5,3,1
3,4,28,1,1,3,0
4,5,74,1,5,?,1
...,...,...,...,...,...,...
956,4,47,2,1,3,0
957,4,56,4,5,3,1
958,4,64,4,5,3,0
959,5,66,4,5,3,1


#### Устранение строк с пропущенными значениями

In [22]:
df.replace('?', np.nan, inplace=True)
df.isnull().sum()

0     2
1     5
2    31
3    48
4    76
5     0
dtype: int64

In [23]:
df.dropna(inplace=True)
df

# Замена NaN на среднее значение по столбцу
#imr = SimpleImputer(missing_values=np.nan, strategy='mean')
#imr = imr.fit(df)
#imputed_data = imr.transform(df.values)
#df = pd.DataFrame.from_dict(imputed_data)
#df

,0,1,2,3,4,5
0,5,67,3,5,3,1
2,5,58,4,5,3,1
3,4,28,1,1,3,0
8,5,57,1,5,3,1
10,5,76,1,4,3,1
...,...,...,...,...,...,...
956,4,47,2,1,3,0
957,4,56,4,5,3,1
958,4,64,4,5,3,0
959,5,66,4,5,3,1


#### Разделение данных на тренировочный и тестовый наборы (70:30)

In [24]:
X = df.iloc[:, 1:4].values
y = df.iloc[:, 5].values

if Version(sklearn_version) < '0.18':
    from sklearn.cross_validation import train_test_split
else:
    from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)


#### Стандартизация признаков

In [25]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

## Персептрон

In [26]:
from sklearn.linear_model import Perceptron

myPerceptron = Perceptron(max_iter=40, eta0=0.1, random_state=0)
myPerceptron.fit(X_train_std, y_train)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=0.1,
           fit_intercept=True, max_iter=40, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)

In [27]:
y_pred = myPerceptron.predict(X_test_std)
print('Число ошибочно классифицированных выборок: %d' % (y_test != y_pred).sum())

Число ошибочно классифицированных выборок: 65


In [28]:
from sklearn.metrics import accuracy_score, recall_score, precision_score

print('Полнота (recall) для каждого класса: ', end = '')
print(recall_score(y_test, y_pred, average = None))
print('Точность (precision) для каждого класса: ', end = '')
print(precision_score(y_test, y_pred, average = None))

print('Верность (accuracy) всей модели: %.2f' % accuracy_score(y_test, y_pred))

Полнота (recall) для каждого класса: [0.73282443 0.74576271]
Точность (precision) для каждого класса: [0.76190476 0.71544715]
Верность (accuracy) всей модели: 0.74


## Случайный лес

In [29]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(criterion='entropy',
                                n_estimators=10, 
                                random_state=1,
                                n_jobs=2)
forest.fit(X_train_std, y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
                       oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [30]:
y_pred = forest.predict(X_test_std)
print('Число ошибочно классифицированных выборок: %d' % (y_test != y_pred).sum())

Число ошибочно классифицированных выборок: 62


In [31]:
print('Полнота (recall) для каждого класса: ', end = '')
print(recall_score(y_test, y_pred, average = None))
print('Точность (precision) для каждого класса: ', end = '')
print(precision_score(y_test, y_pred, average = None))

print('Верность (accuracy) всей модели: %.2f' % accuracy_score(y_test, y_pred))

Полнота (recall) для каждого класса: [0.7480916  0.75423729]
Точность (precision) для каждого класса: [0.77165354 0.7295082 ]
Верность (accuracy) всей модели: 0.75


## К-ближайших соседей

In [32]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
knn.fit(X_train_std, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [33]:
y_pred = knn.predict(X_test_std)
print('Число ошибочно классифицированных выборок: %d' % (y_test != y_pred).sum())

Число ошибочно классифицированных выборок: 55


In [34]:
print('Полнота (recall) для каждого класса: ', end = '')
print(recall_score(y_test, y_pred, average = None))
print('Точность (precision) для каждого класса: ', end = '')
print(precision_score(y_test, y_pred, average = None))

print('Верность (accuracy) всей модели: %.2f' % accuracy_score(y_test, y_pred))

Полнота (recall) для каждого класса: [0.75572519 0.80508475]
Точность (precision) для каждого класса: [0.81147541 0.7480315 ]
Верность (accuracy) всей модели: 0.78
